In [1]:
import pandas as pd
import numpy as np
import shutil
import os

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout

In [3]:
def copy_image(source_path, destination_path):
    try:
        shutil.copy(source_path, destination_path)
        print("Done")
    except FileNotFoundError:
        print("Not Found")
    except PermissionError:
        print("No Permission")

In [4]:
import keras.utils as image
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range = 30,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.2,
    zoom_range = [0.6,1.4],
    channel_shift_range = 0.0,
    fill_mode = 'constant',
    cval = 0.0,
    horizontal_flip = 0.5,
    vertical_flip = 0.5,
    rescale = 1./255,
    data_format = None,
    validation_split = 0.05,
    interpolation_order = 1,
    dtype = None,
)

In [5]:
import cv2
data_folder = r"C:\Users\dhruv\Downloads\ConstellationDatasetSplit"
train_folder = os.path.join(data_folder, "train")

augmented_folder = os.path.join(data_folder, "aug_data")
mapper_file = os.path.join(data_folder, "mapper.xlsx")
mapper_df = pd.read_excel(mapper_file)

if not os.path.exists(augmented_folder):
    os.makedirs(augmented_folder)


In [6]:
for filename in os.listdir(train_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(train_folder, filename)
        constellation_label = mapper_df[mapper_df["Constellation Image file"] == filename]["Constellation name"].values[0]

        constellation_folder = os.path.join(augmented_folder, constellation_label)

        if not os.path.exists(constellation_folder):
            os.makedirs(constellation_folder)
            new_image_path = os.path.join(constellation_folder, filename)
            copy_image(image_path,new_image_path)
        img = image.load_img(image_path,target_size=(300,300))
        img = image.img_to_array(img)
        padded_image = cv2.copyMakeBorder(img, 100, 100, 100, 100, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        #check about 100
        
        i = 0
        input_batch = padded_image.reshape(1,500,500,3)
        for output in datagen.flow(input_batch,batch_size = 1,
                                   save_to_dir = constellation_folder):
            i = i + 1
            if i == 200:
                break

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [7]:
train_datagen = ImageDataGenerator()
# Flow training images in batches
train_generator = train_datagen.flow_from_directory(
        r"C:\Users\dhruv\Downloads\ConstellationDatasetSplit\aug_data",
        target_size = (500, 500),
        batch_size = 88,
        # Binary labels for crossentropy
        class_mode = 'categorical')

Found 17515 images belonging to 88 classes.


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.33),
   layers.Dense(88, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
dataset_dir = "C:/Users/dhruv/Downloads/ConstellationDatasetSplit/aug_data"

# Define batch size and image dimensions
batch_size = 32
image_size = (224, 224)

# Create data generators for training and validation
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify 'training'
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify 'validation'
)

num_train_samples = train_generator.samples
num_validation_samples = validation_generator.samples
steps_per_epoch = num_train_samples // batch_size
validation_steps = num_validation_samples // batch_size

# Train the model 
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=3, 
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Save
model.save('model.h5')

Found 16686 images belonging to 88 classes.
Found 829 images belonging to 88 classes.
Epoch 1/3
 72/521 [===>..........................] - ETA: 14:40 - loss: 4.4715 - accuracy: 0.0122